In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
games = pd.read_csv('master_data/odi_ballbyball.csv')
info = pd.read_csv('master_data/odi_info.csv')

In [3]:
print games.columns
games.head()

Index([u'game_id', u'innings', u'batting_side', u'over_number',
       u'ball_in_over', u'bowler', u'batsman', u'non_striker', u'batted_runs',
       u'xb', u'xlb', u'xnb', u'xw', u'xpen', u'total_extras', u'total_runs',
       u'non_boundary', u'wicket', u'kind', u'player_out', u'fielders'],
      dtype='object')


,game_id,innings,batting_side,over_number,ball_in_over,bowler,batsman,non_striker,batted_runs,xb,...,xnb,xw,xpen,total_extras,total_runs,non_boundary,wicket,kind,player_out,fielders
0,656441,1st innings,England,0,1,SL Malinga,MM Ali,IR Bell,0,NaN,...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN
1,656441,1st innings,England,0,2,SL Malinga,MM Ali,IR Bell,0,NaN,...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN
2,656441,1st innings,England,0,3,SL Malinga,MM Ali,IR Bell,1,NaN,...,NaN,NaN,NaN,0,1,0,NaN,NaN,NaN,NaN
3,656441,1st innings,England,0,4,SL Malinga,IR Bell,MM Ali,0,NaN,...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN
4,656441,1st innings,England,0,5,SL Malinga,IR Bell,MM Ali,0,NaN,...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN


In [4]:
games.wicket.value_counts()

1    13860
dtype: int64

In [5]:
def do_cumsums(df):
    df['inn_runs'] = df['total_runs'].cumsum()
    df['inn_wickets'] = df['wicket'].fillna(0).cumsum()
    return df
games = games.groupby(['game_id','innings']).apply(do_cumsums)

In [6]:
games_trim = games[['game_id','innings','batting_side','over_number','ball_in_over','inn_runs','inn_wickets']]

In [7]:
info.head()

,game_id,date,venue,city,neutral,match_type,overs,team_inn1,team_inn2,toss_winner,toss_decision,match_winner,win_type,margin,umpires,man_of_match
0,656441,2015-03-01,Westpac Stadium,Wellington,1,ODI,50,England,Sri Lanka,England,bat,Sri Lanka,wickets,9,BNJ Oxenford|RJ Tucker,KC Sangakkara
1,656469,2015-03-12,Westpac Stadium,Wellington,1,ODI,50,South Africa,United Arab Emirates,United Arab Emirates,field,South Africa,runs,146,SJ Davis|RJ Tucker,AB de Villiers
2,452152,2010-06-09,Harare Sports Club,NaN,0,ODI,50,Zimbabwe,Sri Lanka,Sri Lanka,field,Sri Lanka,wickets,9,RE Koertzen|RB Tiffin,TM Dilshan
3,657643,2013-12-25,Sheikh Zayed Stadium,Abu Dhabi,1,ODI,50,Sri Lanka,Pakistan,Sri Lanka,bat,Pakistan,wickets,8,Ahsan Raza|RK Illingworth,Mohammad Hafeez
4,860269,2015-07-11,Rangiri Dambulla International Stadium,NaN,0,ODI,50,Sri Lanka,Pakistan,Pakistan,field,Pakistan,wickets,6,IJ Gould|REJ Martinesz,Mohammad Hafeez


In [8]:
info_trim = info[['game_id','date','match_winner']]

In [9]:
info_trim.match_winner.value_counts()

India                   142
Australia               125
Sri Lanka               111
South Africa             97
Pakistan                 94
England                  86
New Zealand              73
Bangladesh               54
West Indies              50
no result                41
Zimbabwe                 24
Ireland                  18
Afghanistan               9
tie                       9
Kenya                     8
Scotland                  7
Canada                    4
Netherlands               3
Asia XI                   3
United Arab Emirates      3
Bermuda                   1
dtype: int64

In [10]:
ball_info = pd.merge(games_trim, info_trim, on = 'game_id', how = 'left')

In [11]:
ball_info = ball_info[~ball_info.match_winner.isin(['tie','no result'])]

In [12]:
ball_info['bat_win'] = (ball_info.batting_side == ball_info.match_winner)*1

In [13]:
ball_info['diff_over_6'] = ball_info[ball_info.ball_in_over > 6].ball_in_over - 6
ball_info.diff_over_6.fillna(0, inplace = True)

In [14]:
ball_info.head()

,game_id,innings,batting_side,over_number,ball_in_over,inn_runs,inn_wickets,date,match_winner,bat_win,diff_over_6
0,656441,1st innings,England,0,1,0,0,2015-03-01,Sri Lanka,0,0
1,656441,1st innings,England,0,2,0,0,2015-03-01,Sri Lanka,0,0
2,656441,1st innings,England,0,3,1,0,2015-03-01,Sri Lanka,0,0
3,656441,1st innings,England,0,4,1,0,2015-03-01,Sri Lanka,0,0
4,656441,1st innings,England,0,5,1,0,2015-03-01,Sri Lanka,0,0


In [15]:
ball_info['balls_left'] = (300 - ball_info['over_number']*6 - ball_info['ball_in_over'] + ball_info['diff_over_6'])

In [16]:
ball_info.drop(['diff_over_6'], axis = 1, inplace = True)

In [17]:
ball_info['pct_balls_left'] = ball_info.balls_left / 300
ball_info['pct_wickets_left'] = (10-ball_info.inn_wickets)/10

In [18]:
ball_info.head()

,game_id,innings,batting_side,over_number,ball_in_over,inn_runs,inn_wickets,date,match_winner,bat_win,balls_left,pct_balls_left,pct_wickets_left
0,656441,1st innings,England,0,1,0,0,2015-03-01,Sri Lanka,0,299,0.996667,1
1,656441,1st innings,England,0,2,0,0,2015-03-01,Sri Lanka,0,298,0.993333,1
2,656441,1st innings,England,0,3,1,0,2015-03-01,Sri Lanka,0,297,0.990000,1
3,656441,1st innings,England,0,4,1,0,2015-03-01,Sri Lanka,0,296,0.986667,1
4,656441,1st innings,England,0,5,1,0,2015-03-01,Sri Lanka,0,295,0.983333,1


In [19]:
# ball_info.to_csv('balls_with_info.csv',index = False)

In [20]:
ball_info.ball_in_over.value_counts()

1     81326
2     81125
3     80950
4     80757
5     80573
6     80371
7     10897
8      1447
9       182
11        5
12        1
dtype: int64

In [21]:
# ball_info.groupby('balls_left').resources_left.mean().sort_index(ascending = False).plot()

In [22]:
chase_numbers = ball_info[ball_info.innings == '1st innings'].groupby('game_id').inn_runs.max().reset_index()
chase_numbers.columns = ['game_id','chase_total']
balls_w_chase = pd.merge(ball_info, chase_numbers, on = 'game_id', how = 'left')

In [23]:
balls_w_chase.chase_total[balls_w_chase.innings == '1st innings'] = None

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
balls_w_chase['runs_to_chase'] = balls_w_chase.chase_total - balls_w_chase.inn_runs

In [25]:
balls_w_chase.runs_to_chase[balls_w_chase.runs_to_chase < 0] = 0

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [26]:
balls_w_chase.tail()

,game_id,innings,batting_side,over_number,ball_in_over,inn_runs,inn_wickets,date,match_winner,bat_win,balls_left,pct_balls_left,pct_wickets_left,chase_total,runs_to_chase
497629,446958,2nd innings,Australia,36,6,206,9,2010-11-05,Sri Lanka,0,78,0.260000,0.1,213,7
497630,446958,2nd innings,Australia,37,1,207,9,2010-11-05,Sri Lanka,0,77,0.256667,0.1,213,6
497631,446958,2nd innings,Australia,37,2,207,9,2010-11-05,Sri Lanka,0,76,0.253333,0.1,213,6
497632,446958,2nd innings,Australia,37,3,210,9,2010-11-05,Sri Lanka,0,75,0.250000,0.1,213,3
497633,446958,2nd innings,Australia,37,4,210,10,2010-11-05,Sri Lanka,0,74,0.246667,0.0,213,3


In [27]:
balls_w_chase.head()

,game_id,innings,batting_side,over_number,ball_in_over,inn_runs,inn_wickets,date,match_winner,bat_win,balls_left,pct_balls_left,pct_wickets_left,chase_total,runs_to_chase
0,656441,1st innings,England,0,1,0,0,2015-03-01,Sri Lanka,0,299,0.996667,1,NaN,NaN
1,656441,1st innings,England,0,2,0,0,2015-03-01,Sri Lanka,0,298,0.993333,1,NaN,NaN
2,656441,1st innings,England,0,3,1,0,2015-03-01,Sri Lanka,0,297,0.990000,1,NaN,NaN
3,656441,1st innings,England,0,4,1,0,2015-03-01,Sri Lanka,0,296,0.986667,1,NaN,NaN
4,656441,1st innings,England,0,5,1,0,2015-03-01,Sri Lanka,0,295,0.983333,1,NaN,NaN
